<a href="https://colab.research.google.com/github/pearl-yu/twitch_project/blob/main/10102022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Run this line if using Colab.
!git clone https://github.com/pearl-yu/twitch_project
%cd twitch_project/

Cloning into 'twitch_project'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 59 (delta 23), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (59/59), done.
/content/twitch_project/twitch_project


In [ ]:
import requests  # get the content from the API by using the get() method. json() method converts the API response to JSON format for easy handling.
import json   # to work with the returned json content from API.
import numpy as np
import pandas as pd

from pandas.io.json import json_normalize

import matplotlib.pyplot as plt

from datetime import datetime
import re

In [ ]:
## Pearl's credentials
client_id = 'tamrylbvlu3wub4r8c5fddwsou246x'
client_secret= 'y4enh38kpiwkh7j00nqvv185ttsedw'

# Defining the URL to which we will make the request
url = 'https://id.twitch.tv/oauth2/token'
# Defining the parameters to be passed to the request
body = {
  'client_id'     : client_id,
  'client_secret' : client_secret,
  'grant_type'    : 'client_credentials'
}

access_code = requests.post(url, params=body) # Making a POST request to the URL to retrieve the access token
access_token = json.loads(access_code.text) #access token response is a JSON-encoded app access token

access_token = access_token['access_token']

With the sample:
- Pull user ids by calling the https://dev.twitch.tv/docs/api/reference#get-users endpoint, specifying the login parameter.
- Pull the video ids following this: https://github.com/btjmga/twitch-getallvods  (I didn't have the code to pull ALL the vids)
- Pull the chatfiles of these videos. (using the downloader https://github.com/lay295/TwitchDownloader is easy. Check  https://github.com/Chaparro/rechat-dl and https://github.com/malciin/twitch-comments-downloader for code.)
- Construct variables from the chatfile.
- Call what other stuff? 

In [ ]:
path = './data/sample_by_growing_sullygnome/Fastest growing Twitch streamers, past 30 days - SullyGnome.csv'
df = pd.read_csv(path)

df.head()

,Unnamed: 0,Unnamed: 1,Channel,Watch time (mins),Stream time (mins),Peak viewers,Average viewers,Followers,Followers gained,Partnered,Mature,Language,Unnamed: 12
0,1,https://static-cdn.jtvnw.net/jtv_user_pictures...,KaiCenat,750515550,13845,174509,54208,2402017,346668,True,False,English,KaiCenat
1,2,https://static-cdn.jtvnw.net/jtv_user_pictures...,ElSpreen,318862545,5505,173221,57922,5618251,230563,True,False,Spanish,ElSpreen
2,3,https://static-cdn.jtvnw.net/jtv_user_pictures...,PaulinhoLOKObr,412465830,6075,141526,67895,1235167,222035,True,False,Portuguese,PaulinhoLOKObr
3,4,https://static-cdn.jtvnw.net/jtv_user_pictures...,WestCOL,172343475,5460,145907,31564,863854,212347,True,False,Spanish,WestCOL
4,5,https://static-cdn.jtvnw.net/jtv_user_pictures...,SypherPK,134891265,14730,151092,9157,6213265,205617,True,False,English,SypherPK


In [ ]:
def api_info(url = 'https://api.twitch.tv/helix/users?id=', key_series = df['Channel'], n = 5):
    
    headers = {
            'Authorization' : 'Bearer '+str(access_token),
            'Client-Id' : client_id
        }

    temp_df = pd.DataFrame()
    for i in range(0,5) :
        temp_response = requests.get(url+str(key_series[i]), headers=headers)
        # Load the JSON
        temp_response_json = json.loads(temp_response.text)
        if list(temp_response_json.keys())[0] == 'error':
            continue
        else:
            temp_data = temp_response_json['data']
            temp_df_temp = pd.DataFrame.from_dict(json_normalize(temp_data), orient='columns')
        
            frames = [temp_df, temp_df_temp]
            temp_df = pd.concat(frames, ignore_index=True)
    
    print(temp_df.shape)
    print(temp_df.columns)
    
    return temp_df

In [ ]:
## call the get_users end points to retrieve user id
user_df = api_info(url = 'https://api.twitch.tv/helix/users?login=',
         key_series = df['Channel'], 
         n = 5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


(5, 10)
Index(['id', 'login', 'display_name', 'type', 'broadcaster_type',
       'description', 'profile_image_url', 'offline_image_url', 'view_count',
       'created_at'],
      dtype='object')


In [ ]:
## call the get_videos end points to retrieve all video ids of the users
video_df = api_info(url = 'https://api.twitch.tv/helix/videos?user_id=',
         key_series = user_df['id'], 
         n = 5)

video_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


(100, 17)
Index(['id', 'stream_id', 'user_id', 'user_login', 'user_name', 'title',
       'description', 'created_at', 'published_at', 'url', 'thumbnail_url',
       'viewable', 'view_count', 'language', 'type', 'duration',
       'muted_segments'],
      dtype='object')


,id,stream_id,user_id,user_login,user_name,title,description,created_at,published_at,url,thumbnail_url,viewable,view_count,language,type,duration,muted_segments
0,1620026122,47280134781,641972806,kaicenat,KaiCenat,🎃THE DWARF HAS RETURNED🎃CLICK HERE NOW🎃CALLING...,,2022-10-10T00:10:28Z,2022-10-10T00:10:28Z,https://www.twitch.tv/videos/1620026122,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9...,public,2449380,en,archive,10h6m42s,None
1,1610443346,47258864461,641972806,kaicenat,KaiCenat,🟧AHHH FINAL DAY OF STREAMING🟧THE TALLEST DWARF...,,2022-10-06T00:15:06Z,2022-10-06T00:15:06Z,https://www.twitch.tv/videos/1610443346,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9...,public,2142479,en,archive,8h11m30s,None
2,1609543460,47253862157,641972806,kaicenat,KaiCenat,🟥MAFIA GET IN HERE NOW🟥TONIGHT WILL BE LIT🟥#1 ...,,2022-10-05T00:18:37Z,2022-10-05T00:18:37Z,https://www.twitch.tv/videos/1609543460,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9...,public,1816939,en,archive,7h40m58s,None
3,1608653502,47248869229,641972806,kaicenat,KaiCenat,🟪THE BIGGEST DWARF IS BACK🟪CLICK HERE NOW🟪CALL...,,2022-10-04T00:04:49Z,2022-10-04T00:04:49Z,https://www.twitch.tv/videos/1608653502,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9...,public,2135369,en,archive,9h20m6s,None
4,1606878570,47238591133,641972806,kaicenat,KaiCenat,🟩BOYZ NIGHTS IS HERE🟩TONIGHT IS FOR THE BOYZ🟩C...,,2022-10-02T00:47:20Z,2022-10-02T00:47:20Z,https://www.twitch.tv/videos/1606878570,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9...,public,1920199,en,archive,6h5m32s,None


In [ ]:
video_df['url'][0]

'https://www.twitch.tv/videos/1620026122'

In [ ]:
## retrieve chat files of these videos.

In [ ]:
# https://pypi.org/project/chat-downloader/
!pip install chat-downloader
from chat_downloader import ChatDownloader

In [ ]:
url = video_df['url'][0]
chat = ChatDownloader().get_chat(url)       # create a generator
         # print the formatted message

In [ ]:
# For each chat message, this is the full structure
next(chat)

{'message_id': '333e49e0-ec2e-4091-9ead-debad8daf6e3',
 'timestamp': 1665360629705000,
 'time_in_seconds': 1.005,
 'author': {'display_name': 'irocki',
  'id': '594447720',
  'name': 'irocki',
  'type': 'user',
  'bio': None,
  'created_at': 1602378852556209,
  'images': [{'url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/bab962cc-3fc7-4a18-b746-e17545a61128-profile_image-300x300.png',
    'width': 300,
    'height': 300,
    'id': '300x300'},
   {'url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/bab962cc-3fc7-4a18-b746-e17545a61128-profile_image-70x70.png',
    'width': 70,
    'height': 70,
    'id': '70x70'}],
  'badges': [{'name': 'subscriber',
    'version': 3,
    'title': '3-Month Subscriber',
    'description': '3-Month Subscriber',
    'click_action': 'subscribe_to_channel',
    'click_url': '',
    'icons': [{'url': 'https://static-cdn.jtvnw.net/badges/v1/92ebb970-6200-48ab-a96a-8add2d4a4e8d/1',
      'width': 18,
      'height': 18,
      'id': '18x18'},
     {'url

In [ ]:
## Now what do we want to put in a dataframe? 
url = video_df['url'][0]
chat = ChatDownloader().get_chat(url)   

timestamp = []
time_in_seconds = []
messages = []
author_display_name = []
# author_id = []
## Let's hold off the badges and emotes. Those are multi-levels. But they'll be super helpful!!

import itertools

top5 = itertools.islice(chat, 5)

for message in top5:                        # iterate over messages
    timestamp.append(message['timestamp'])
    time_in_seconds.append(message['time_in_seconds'])
    messages.append(message['message'])
    author_display_name.append(message['author']['display_name'])
    ## better way to flatten. 
    ## can be zipped to a function

In [ ]:
chat_df = pd.DataFrame(list(zip(timestamp,time_in_seconds,messages,author_display_name)),
                       columns = ['timestamp','time_in_seconds','messages','author_display_name'])

In [ ]:
chat_df.head()
## There're emotes! 

,timestamp,time_in_seconds,messages,author_display_name
0,1665360629212000,0.512,🔥🔥🔥🔥🔥🔥,jaytwinem
1,1665360629492000,0.792,WWWWWW,MimiGutta
2,1665360629705000,1.005,kaicENAT kaicENAT kaicENAT kaicENAT kaicENAT k...,irocki
3,1665360629820000,1.120,WW,Exstaticss
4,1665360629829000,1.129,WYAT STFU,JordanFeelss


In [ ]:
#for message in chat:                        # iterate over messages. It's not json!! It's j
#    print(json.dumps(message, indent = 4, sort_keys=True))

Parse

In [ ]:
# No need for this anymore.
## parse chat file
def get_chat_dataframe(file):
    data = []
    with open(file, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')
        for line in lines:
            #print(line)
            
            time_logged = line.split('UTC')[0][1:].strip()
            #time_logged = datetime.strptime(time_logged,'%Y-%m-%d-%H:%M:%S')
            #print(time_logged)
            
            username_message = line.split(']')[1:]
            username_message = '—'.join(username_message).strip()
            #print(username_message)
            try:
                temp = username_message.split(":")
                username = temp[0]
                message = temp[1]
            except IndexError:
                username = temp[0]
                message = '-'
            #print(username)
            #print(message)
            
            d = {'dt': time_logged,
                 'username': username,
                 'message': message
                    }
            data.append(d)
            
    return pd.DataFrame().from_records(data)

In [ ]:
## This is an example
chat_df = get_chat_dataframe('[1-21-22] Mongraal - Winning Duo Cash Cup With FaZe BL GUILD Flikk (Webcam Mic) - Chat.txt')

chat_df.head()

Cleaning and preprocessing

In [ ]:
!pip install texthero

In [ ]:
## cleaning
import texthero as hero
from texthero import preprocessing

KeyError: ignored

In [ ]:
## customize a pipelifne
custom_pipeline = [preprocessing.lowercase, 
                   preprocessing.remove_punctuation,
                   preprocessing.remove_urls]

chat_df_temp = hero.clean(chat_df['message'])

NameError: ignored

In [ ]:
# Custom STOPWORDS removal -- Need to think about this.Could use the cloud as below 
from texthero import stopwords
default_stopwords = stopwords.DEFAULT
custom_stopwords = default_stopwords.union(set(['yo','play',"twitch","make", "use", "thank", "content", "good", "use",
                                                "think", "need", "harrisheller", "like", "stream",
                                               "kekw","catjam","tim","timthetatman","tatkevinh",
                                               "wipz","docspin","pog","tatlove","lol","lul","omegalul",
                                               "biblethump","clap","tathypers","pepeja","kappa","tattopd","ppsmoke",
                                               "pepelaugh","gopackgo","gachihyper","tatkevinh", "wipz",
                                               "pausechamp","yep","lmao","jack","lulw","monkaw","kreygasm",
                                               "pepega","peped","foxsalt","pogchamp","xqcn","get","back",
                                               "tattuff","tatfat","tatpumpkin","lmao","sadge","sippy",
                                               "pogu","poggers","consolecd","widepeepohappy","pogu","tategg2",
                                               "modcheck","timmy","tathmm","tats","got","com","babyrage",
                                               "xqcp","tatw","pokiw","know", "thats","pepocd","tatafk",
                                               "4weird","tatkkevin", "tatblanket","tatglam","tategg1","wutface",
                                               "blobdance", "kapp","tatbruh","kappapride","facebaby","xqc",
                                               "xqcm","bora","hyperclap","tatlit","5head","gachibass", "go", "ur",
                                                "yes","going","would","im","oh","dez","taty","tk","u","sg", "dont",
                                                "hey","hf","look","anita","anitaheart","leeper","anitahey",
                                                "anitadab","anitadoubt","anitahands", "anitalul","anitapog","anitasword",
                                                "anitaprime","anitafeels","residentsleeper","anitapride","anital",
                                                "anitabiscuit", "thing","widepeeposad","anitahype","among", "lesbianpride",
                                                "heyguys","seemsgood","notlikethis","hi","also","much","yeah",
                                               ])) ## Add as per requirement
# data = hero.remove_stopwords(data, default_stopwords)
chat_df_clean = hero.remove_stopwords(chat_df_temp, custom_stopwords)

In [ ]:
## visualize with wordcloud
hero.visualization.wordcloud(data, font_path = None, width = 400, height = 200, max_words=200, 
                             mask=None, contour_width=0, 
                             contour_color='PAPAYAWHIP', background_color='WHITE', 
                             relative_scaling='auto', colormap=None, return_figure=False)

In [ ]:
import gensim

# Tokenize
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(chat_df_clean))

print(data_words[:5])

In [ ]:
import spacy

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adjective, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:50])

LDA

In [ ]:
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
# !pip install -U pyLDAvis
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Word-Document Matrix
# This converts a collection of text documents to a matrix of token counts. 
# A document-term matrix or term-document matrix is a mathematical matrix that describes the frequency of terms that occur in a collection of documents. 
# In a document-term matrix, rows correspond to documents in the collection and columns correspond to terms. 

vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum number occurences of a word required
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}')  # num of characters > 3
                            

data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [ ]:
# Build LDA model with Sklearn

lda_model = LatentDirichletAllocation(n_components=15,               # Number of topics
                                      max_iter=10,                   # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,              # Random state
                                      batch_size=128)                # n docs in each learning iter
                                            
                                      
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

In [ ]:
#Evaluate model quality

# Log Likelihood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp((-1) * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

In [ ]:
# Grid Search for Best LDA Model
# Grid search is a tuning technique that attempts to compute the optimum values of hyperparameters. 
# It is an exhaustive search that is performed on a the specific parameter values of a model. The model is also known as an estimator.
# This is computationally expensive and usually takes time... 

from sklearn.model_selection import GridSearchCV

# Define Search Param
params = {'n_components': [5, 10, 15,20], 'learning_decay': [.5, .7, .9]}

# Model
lda = LatentDirichletAllocation()

# Grid Search
model = GridSearchCV(lda, param_grid=params)

# Perform Grid Search
model.fit(data_vectorized)

In [ ]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

In [ ]:
# Reflections:
# how important and how frequent certain words are in the corpus - what themes/etc. may emerge. 
# Show top n keywords for each topic

def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=10)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords